In [3]:
import mysql.connector
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200

# silahkan diinstall jika ada library yang belum terinstall

In [4]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = '',
    database = 'week13_db'
)

# disesuaikan dengan user dan password SQL masing masing 

In [5]:
mycursor = mydb.cursor()

# create function
def sql(yourQuery):
    mycursor.execute(yourQuery)
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult, columns = mycursor.column_names)
    return df

In [6]:
# trying function
sql(
    '''
    SELECT *
    FROM sales;
    '''
)

,id,name,product,quantity,sales_date
0,1,Jackson,Keyboard,4,2021-01-01
1,2,Louis,Keyboard,2,2021-01-01
2,3,Kevin,Printer,8,2021-01-01
3,4,Louis,Smartphone,1,2021-01-01
4,5,Jackson,Smartphone,9,2021-01-01
...,...,...,...,...,...
3261,3262,Rich,Tablet,9,2023-12-30
3262,3263,Kevin,Keyboard,5,2023-12-30
3263,3264,Rich,Tablet,4,2023-12-30
3264,3265,Rich,Monitor,6,2023-12-31


In [24]:
def monthly_sales():
    query = '''
    SELECT 
        product,
        EXTRACT(YEAR FROM sales_date) AS year,
        DATE_FORMAT(sales_date, '%M') AS month_text,
        SUM(quantity) AS total_quantity
    FROM sales
    GROUP BY product, year, EXTRACT(MONTH FROM sales_date), DATE_FORMAT(sales_date, '%M')
    ORDER BY year, EXTRACT(MONTH FROM sales_date);
    '''
    return sql(query)

df_monthly_sales = monthly_sales()
print(df_monthly_sales)

        product  year month_text total_quantity
0    Smartphone  2021    January             98
1       Monitor  2021    January             81
2        Tablet  2021    January             65
3      Keyboard  2021    January             83
4         Mouse  2021    January             65
..          ...   ...        ...            ...
247     Printer  2023   December             68
248  Smartphone  2023   December             66
249      Laptop  2023   December             47
250     Monitor  2023   December             54
251    Keyboard  2023   December             64

[252 rows x 4 columns]


In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Fetch data if not loaded
df_sales = monthly_sales()

def plot_sales(p, y, m):
    df = df_sales.copy()
    if p != 'All': df = df[df['product'] == p]
    if y != 'All': df = df[df['year'] == int(y)]
    if m != 'All': df = df[df['month'] == int(m)]
    
    if df.empty:
        print("No data available for selected filters.")
        return
    
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str))
    plt.figure(figsize=(10, 5))
    for key, grp in df.groupby('product'):
        plt.plot(grp['date'], grp['total_quantity'], label=key, marker='o')
    
    plt.title('Monthly Sales by Product')
    plt.xlabel('Date')
    plt.ylabel('Total Quantity')
    plt.grid(True)
    plt.legend(title='Product')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Widgets
p_w = widgets.Dropdown(options=['All'] + sorted(df_sales['product'].unique()), description='Product:', value='All')
y_w = widgets.Dropdown(options=['All'] + sorted(df_sales['year'].astype(str).unique()), description='Year:', value='All')
m_w = widgets.Dropdown(options=['All'] + sorted(df_sales['month'].astype(str).unique()), description='Month:', value='All')

# Widget interaction setup
ui = widgets.HBox([p_w, y_w, m_w])
out = widgets.interactive_output(plot_sales, {'p': p_w, 'y': y_w, 'm': m_w})

# Display the interface
display(ui, out)

Output()

In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

def plot_sales_with_forecast(p):
    df = df_sales.copy()
    if p != 'All':
        df = df[df['product'] == p]
    
    if df.empty:
        print("No data available for selected filters.")
        return
    
    # Convert year and month into a datetime for plotting and modeling
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str))
    plt.figure(figsize=(12, 6))
    
    # Prepare the data for the regression model
    df['date_ordinal'] = df['date'].apply(lambda x: x.toordinal())
    
    # Fit the regression model
    if not df['product'].empty:
        X = df[['date_ordinal']]
        y = df['total_quantity']
        model = LinearRegression().fit(X, y)
        
        # Predict for 2024
        future_dates = pd.date_range('2024-01-01', '2024-12-01', freq='MS')
        future_dates_ordinal = [d.toordinal() for d in future_dates]
        future_preds = model.predict(np.array(future_dates_ordinal).reshape(-1,1))
        
        # Plot historical data
        plt.plot(df['date'], df['total_quantity'], label=f"{p} Actual", marker='o')
        # Plot predictions
        plt.plot(future_dates, future_preds, label=f"{p} Forecast", linestyle='--')
    
    plt.title(f'Monthly Sales Forecast for 2024 - {p}')
    plt.xlabel('Date')
    plt.ylabel('Total Quantity')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Widgets for product filtering
product_widget = widgets.Dropdown(options=['All'] + sorted(df_sales['product'].unique()), description='Product:', value='All')
# Widget interaction setup
out = widgets.interactive_output(plot_sales_with_forecast, {'p': product_widget})
# Display the interface
display(product_widget, out)

Dropdown(description='Product:', options=('All', 'Keyboard', 'Laptop', 'Monitor', 'Mouse', 'Printer', 'Smartph…

Output()

In [ ]:
# 